In [ ]:
# mount drive
from google.colab import drive
import os
drive.mount("/content/drive")
os.chdir("drive/My Drive")

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime
import copy

## Data Manipulation

**Purpose of this notebook:** In this notebook, we select weather data from 2015-2020. We also increase the grid size by enlarging the distance between the unique values for latitude and longitude. The ultimate goal of this is to increase the granularity of our data to limit its size.

## 1. Load and slice the data

In [ ]:
# load .nc file 
weather = xr.open_mfdataset("weather.nc")
weather

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


<xarray.Dataset>
Dimensions:    (latitude: 99, longitude: 232, time: 3652)
Coordinates:
  * longitude  (longitude) float32 -124.78 -124.53 -124.28 ... -67.28 -67.03
  * latitude   (latitude) float32 49.24 48.99 48.74 48.49 ... 25.24 24.99 24.74
  * time       (time) datetime64[ns] 2010-01-01T12:00:00 ... 2019-12-31T12:00:00
Data variables:
    d          (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    cc         (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    z          (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    o3         (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    pv         (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    r          (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    ciwc       (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    clwc       (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    q          (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    crwc       (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    cswc       (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    t          (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    u          (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    v          (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    w          (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
    vo         (time, latitude, longitude) float32 dask.array<chunksize=(3652, 99, 232), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-04-27 19:17:13 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [ ]:
# convert database to pandas dataframe
weather = weather.to_dataframe()
weather.reset_index(inplace=True)

In [ ]:
# select data after 2015-01-01
weather_new = weather[weather["time"] > "2015-01-01 12:00:00"]

In [ ]:
# clear memory
del weather
import gc
gc.collect()

260

In [ ]:
# reset index of weather_new
weather_new.reset_index(inplace = True, drop = True)

## 2. Enlarge the grid size
- Note: Each degree of latitude (1.0) is approximately 69 miles

In [ ]:
# define new range (new_range instead of 0.24)
new_range = 0.84

# latitude new range 
lat_range = np.arange(weather_new.latitude.unique().min(), weather_new.latitude.unique().max(), new_range)

# longitude new range
lon_range = np.arange(weather_new.longitude.unique().min(), weather_new.longitude.unique().max(), new_range)

In [ ]:
# create chunks of the data to be able to run it (memory issues)
chunk_size = int(len(weather_new)/6)

In [ ]:
# chunk 1
weather_1 = copy.deepcopy(weather_new.loc[:chunk_size,:])

# lat: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_1["latitude"]
bb = lat_range
weather_1["latitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

# long: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_1["longitude"]
bb = lon_range
weather_1["longitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

In [ ]:
# chunk 2
weather_2 = copy.deepcopy(weather_new.loc[chunk_size:chunk_size*2,:])

# lat: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_2["latitude"]
bb = lat_range
weather_2["latitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

# long: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_2["longitude"]
bb = lon_range
weather_2["longitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

In [ ]:
# chunk 3
weather_3 = copy.deepcopy(weather_new.loc[chunk_size*2:chunk_size*3,:])

# lat: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_3["latitude"]
bb = lat_range
weather_3["latitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

# long: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_3["longitude"]
bb = lon_range
weather_3["longitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

In [ ]:
# chunk 4
weather_4 = copy.deepcopy(weather_new.loc[chunk_size*3:chunk_size*4,:])

# lat: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_4["latitude"]
bb = lat_range
weather_4["latitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

# long: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_4["longitude"]
bb = lon_range
weather_4["longitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

In [ ]:
# chunk 5
weather_5 = copy.deepcopy(weather_new.loc[chunk_size*4:chunk_size*5,:])

# lat: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_5["latitude"]
bb = lat_range
weather_5["latitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

# long: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_5["longitude"]
bb = lon_range
weather_5["longitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

In [ ]:
# chunk 6
weather_6 = copy.deepcopy(weather_new.loc[chunk_size*5:chunk_size*6,:])

# lat: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_6["latitude"]
bb = lat_range
weather_6["latitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

# long: in weather, find closest lat/long value from new range and replace it with that value
aa = weather_6["longitude"]
bb = lon_range
weather_6["longitude"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]

## 3. Drop the duplicates
- Potential for further optimization to increase accuracy: Calculate mean among the new grid and drop all other rows

In [ ]:
weather_1 = weather_1.drop_duplicates(subset=["latitude", "longitude", "time"])

In [ ]:
weather_2 = weather_2.drop_duplicates(subset=["latitude", "longitude", "time"])

In [ ]:
weather_3 = weather_3.drop_duplicates(subset=["latitude", "longitude", "time"])

In [ ]:
weather_4 = weather_4.drop_duplicates(subset=["latitude", "longitude", "time"])

In [ ]:
weather_5 = weather_5.drop_duplicates(subset=["latitude", "longitude", "time"])

In [ ]:
weather_6 = weather_6.drop_duplicates(subset=["latitude", "longitude", "time"])

In [ ]:
weather_final = pd.concat([weather_1, weather_2, weather_3, weather_4, weather_5, weather_6])

In [ ]:
weather_final.reset_index(inplace = True, drop = True)

In [ ]:
weather_final.shape

(4091652, 19)

In [ ]:
weather_final

latitude   longitude  ...         w        vo
0           49.10 -124.779999  ... -0.327145 -0.000049
1           49.10 -124.779999  ...  0.001749 -0.000040
2           49.10 -124.779999  ... -0.291906 -0.000026
3           49.10 -124.779999  ... -0.617445 -0.000069
4           49.10 -124.779999  ... -0.942983 -0.000044
...           ...         ...  ...       ...       ...
4091647     24.74  -67.659999  ...  0.059641 -0.000028
4091648     24.74  -67.659999  ... -0.015871  0.000003
4091649     24.74  -67.659999  ... -0.006641 -0.000031
4091650     24.74  -67.659999  ...  0.106206 -0.000082
4091651     24.74  -67.659999  ... -0.041461 -0.000011

[4091652 rows x 19 columns]

In [ ]:
print(weather_final["latitude"].min(), weather_final["latitude"].max())

24.739999771118164 49.09999977111816


In [ ]:
print(lat_range.min(), lat_range.max())

24.739999771118164 49.09999977111816


In [ ]:
print(weather_final["longitude"].min(), weather_final["longitude"].max())

-124.77999877929688 -67.65999877929664


In [ ]:
print(lon_range.min(), lon_range.max())

-124.77999877929688 -67.65999877929664


In [ ]:
# save as cvs
weather_final.to_csv("weather_final.csv")